<a href="https://colab.research.google.com/github/roshantac/Unet/blob/master/MaskDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!!nvidia-smi

['Sat May 16 11:51:53 2020       ',
 '+-----------------------------------------------------------------------------+',
 '| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |',
 '|-------------------------------+----------------------+----------------------+',
 '| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |',
 '| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |',
 '|===============================+======================+======================|',
 '|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |',
 '| N/A   37C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |',
 '+-------------------------------+----------------------+----------------------+',
 '                                                                               ',
 '+-----------------------------------------------------------------------------+',
 '| Processes:                          

In [2]:
!git clone https://github.com/roshantac/Unet.git

fatal: destination path 'Unet' already exists and is not an empty directory.


In [3]:
!mkdir Unet/data

mkdir: cannot create directory ‘Unet/data’: File exists


In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
!mkdir temp
!cp '/content/drive/My Drive/Eva_DataSet.zip' temp


KeyboardInterrupt: ignored

In [0]:
destDataDir = '/content/temp/'
path_to_zip_file = '/content/temp/Eva_DataSet.zip'
import zipfile
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(destDataDir)

In [0]:
!mv '/content/temp/content/Dataset/Depthmap' '/content/Unet/data/'
!mv '/content/temp/content/Dataset/fgbg' '/content/Unet/data/'
!mv '/content/temp/content/Dataset/fgbgmask' '/content/Unet/data/'

In [0]:
!rm -rf temp

In [3]:
%cd Unet/

/content/Unet


In [0]:
import argparse
import logging
import os
import sys

import numpy as np
import torch
import torch.nn as nn
from torch import optim
from tqdm import tqdm

from eval import eval_net
from unet import UNet

from torch.utils.tensorboard import SummaryWriter
from utils.dataset import BasicDataset
from torch.utils.data import DataLoader, random_split

In [0]:
dir_img = 'data/fgbg/'
dir_mask = 'data/fgbgmask/'
dir_checkpoint = 'checkpoints/'

def train_net(net,
              device,
              epochs=5,
              batch_size=1,
              lr=0.001,
              val_percent=0.1,
              save_cp=True,
              img_scale=0.5):

    dataset = BasicDataset(dir_img, dir_mask, img_scale)
    n_val = int(len(dataset) * val_percent)
    n_train = len(dataset) - n_val
    train, val = random_split(dataset, [n_train, n_val])
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True, drop_last=True)

    writer = SummaryWriter(comment=f'LR_{lr}_BS_{batch_size}_SCALE_{img_scale}')
    global_step = 0

    logging.info(f'''Starting training:
        Epochs:          {epochs}
        Batch size:      {batch_size}
        Learning rate:   {lr}
        Training size:   {n_train}
        Validation size: {n_val}
        Checkpoints:     {save_cp}
        Device:          {device.type}
        Images scaling:  {img_scale}
    ''')

    optimizer = optim.RMSprop(net.parameters(), lr=lr, weight_decay=1e-8, momentum=0.9)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min' if net.n_classes > 1 else 'max', patience=2)
    if net.n_classes > 1:
        criterion = nn.CrossEntropyLoss()
    else:
        criterion = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        net.train()

        epoch_loss = 0
        with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{epochs}', unit='img') as pbar:
            for batch in train_loader:
                imgs = batch['image']
                true_masks = batch['mask']
                assert imgs.shape[1] == net.n_channels, \
                    f'Network has been defined with {net.n_channels} input channels, ' \
                    f'but loaded images have {imgs.shape[1]} channels. Please check that ' \
                    'the images are loaded correctly.'

                imgs = imgs.to(device=device, dtype=torch.float32)
                mask_type = torch.float32 if net.n_classes == 1 else torch.long
                true_masks = true_masks.to(device=device, dtype=mask_type)

                masks_pred = net(imgs)
                loss = criterion(masks_pred, true_masks)
                epoch_loss += loss.item()
                writer.add_scalar('Loss/train', loss.item(), global_step)

                pbar.set_postfix(**{'loss (batch)': loss.item()})

                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_value_(net.parameters(), 0.1)
                optimizer.step()

                pbar.update(imgs.shape[0])
                global_step += 1
                if global_step % (len(dataset) // (10 * batch_size)) == 0:
                    for tag, value in net.named_parameters():
                        tag = tag.replace('.', '/')
                        writer.add_histogram('weights/' + tag, value.data.cpu().numpy(), global_step)
                        writer.add_histogram('grads/' + tag, value.grad.data.cpu().numpy(), global_step)
                    val_score = eval_net(net, val_loader, device)
                    scheduler.step(val_score)
                    writer.add_scalar('learning_rate', optimizer.param_groups[0]['lr'], global_step)

                    if net.n_classes > 1:
                        logging.info('Validation cross entropy: {}'.format(val_score))
                        writer.add_scalar('Loss/test', val_score, global_step)
                    else:
                        logging.info('Validation Dice Coeff: {}'.format(val_score))
                        writer.add_scalar('Dice/test', val_score, global_step)

                    writer.add_images('images', imgs, global_step)
                    if net.n_classes == 1:
                        writer.add_images('masks/true', true_masks, global_step)
                        writer.add_images('masks/pred', torch.sigmoid(masks_pred) > 0.5, global_step)

        if save_cp:
            try:
                os.mkdir(dir_checkpoint)
                logging.info('Created checkpoint directory')
            except OSError:
                pass
            torch.save(net.state_dict(),
                       dir_checkpoint + f'CP_epoch{epoch + 1}.pth')
            logging.info(f'Checkpoint {epoch + 1} saved !')

    writer.close()



In [0]:
net = UNet(n_channels=3, n_classes=1, bilinear=True)

In [7]:
from torchsummary import summary
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.to(device=device)
summary(net, (3,200,200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 200, 200]             896
       BatchNorm2d-2         [-1, 32, 200, 200]              64
              ReLU-3         [-1, 32, 200, 200]               0
            Conv2d-4         [-1, 32, 200, 200]           9,248
       BatchNorm2d-5         [-1, 32, 200, 200]              64
              ReLU-6         [-1, 32, 200, 200]               0
        DoubleConv-7         [-1, 32, 200, 200]               0
         MaxPool2d-8         [-1, 32, 100, 100]               0
            Conv2d-9         [-1, 64, 100, 100]          18,496
      BatchNorm2d-10         [-1, 64, 100, 100]             128
             ReLU-11         [-1, 64, 100, 100]               0
           Conv2d-12         [-1, 64, 100, 100]          36,928
      BatchNorm2d-13         [-1, 64, 100, 100]             128
             ReLU-14         [-1, 64, 1

In [8]:

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

logging.info(f'Using device {device}')

# Change here to adapt to your data
# n_channels=3 for RGB images
# n_classes is the number of probabilities you want to get per pixel
#   - For 1 class and background, use n_classes=1
#   - For 2 classes, use n_classes=1
#   - For N > 2 classes, use n_classes=N

logging.info(f'Network:\n'
              f'\t{net.n_channels} input channels\n'
              f'\t{net.n_classes} output channels (classes)\n'
              f'\t{"Bilinear" if net.bilinear else "Transposed conv"} upscaling')


# faster convolutions, but more memory
# cudnn.benchmark = True
train_net(net=net,
          epochs=1,
          batch_size=128,
          lr=0.1,
          device=device,
          img_scale=1.0,
          val_percent= 10.0/ 100)
# except KeyboardInterrupt:
#     torch.save(net.state_dict(), 'INTERRUPTED.pth')
#     logging.info('Saved interrupt')
#     try:
#         sys.exit(0)
#     except SystemExit:
#         os._exit(0)


INFO: Using device cuda
INFO: Network:
	3 input channels
	1 output channels (classes)
	Bilinear upscaling
INFO: Starting training:
        Epochs:          1
        Batch size:      128
        Learning rate:   0.1
        Training size:   367189
        Validation size: 40798
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1.0
    
Validation round: 100%|██████████| 318/318 [02:38<00:00,  2.39batch/s]
                                                                      INFO: Validation Dice Coeff: 0.3329048146429302
Validation round: 100%|██████████| 318/318 [02:36<00:00,  2.77batch/s]
                                                                      INFO: Validation Dice Coeff: 0.25289381084577095
Validation round: 100%|██████████| 318/318 [02:36<00:00,  2.78batch/s]
                                                                      INFO: Validation Dice Coeff: 0.23635052547514815
Validation round: 100%|██████████| 318/318 [02:37<00:00,  

In [0]:
!cp checkpoints/*.pth '/content/drive/My Drive/Untitled folder/'